In [ ]:
# Primeiro, realizamos a instalação do mlflow e autoML do sklearn

#!pip install auto-sklearn

!pip install --upgrade pip

!pip install mlflow

!pip install tqdm 

In [ ]:
import mlflow
import requests

url = "http://checkip.amazonaws.com"
try:
    response = requests.get(url)
    response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
    IP = response.text.replace('\n', '')
except requests.exceptions.RequestException as e:
    print(f"An error occurred: {e}")

uri_mlflow = "http://"+IP+":8089"
print(uri_mlflow)

mlflow.set_tracking_uri(uri_mlflow)

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# LE O DATASET COMO UM PANDAS DATAFRAME.
df_data = pd.read_csv('curso_dw.csv')

# Definição das colunas que serão features (nota-se que a coluna NOME não está presente)
features = [
    #"MATRICULA",
    "REPROVACOES_MAT_1", 'REPROVACOES_MAT_2', "REPROVACOES_MAT_3", "REPROVACOES_MAT_4",
    "NOTA_MAT_1", "NOTA_MAT_2", "NOTA_MAT_3", "NOTA_MAT_4",
    "INGLES", "H_AULA_PRES", "TAREFAS_ONLINE", "FALTAS", 
    #"INGLES_DESC", # usando todas as colunas
    #"CURSOU_MAT1_DESC", "CURSOU_MAT2_DESC", "CURSOU_MAT3_DESC", "CURSOU_MAT4_DESC",
]

# Definição da variável-alvo
target = ["PERFIL"]

# Preparação dos argumentos para os métodos da biblioteca ``scikit-learn``
X = df_data[features]
y = df_data[target]

# Separação dos dados em um conjunto de treino e um conjunto de teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=337)

In [ ]:
import autosklearn.classification
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import threading
import time


# Parâmetros do AutoML
total_time = 120  # Tempo total (em segundos)
per_model_time = 30  # Tempo máximo por modelo

# Criando uma barra de progresso em thread separada
def show_progress_bar(duration):
    for _ in tqdm(range(duration), desc="AutoML training progress", ncols=100):
        time.sleep(1)

# Iniciando barra de progresso em segundo plano
progress_thread = threading.Thread(target=show_progress_bar, args=(total_time,))
progress_thread.start()

# Treinamento com auto-sklearn
automl = autosklearn.classification.AutoSklearnClassifier(
    time_left_for_this_task=total_time,
    per_run_time_limit=per_model_time,
    tmp_folder='/tmp/autosklearn_tmp',
    #output_folder='/tmp/autosklearn_out',
    disable_evaluator_output=False,
    n_jobs=-1
)

automl.fit(X_train, y_train)

# Espera barra terminar se for mais rápida
progress_thread.join()

# Resultados
print("\Ranking dos modelos testados: \n")
print(automl.leaderboard())

y_pred = automl.predict(X_test)
print(f"\Acurácia: {accuracy_score(y_test, y_pred):.4f}")


In [ ]:
import pandas as pd

from sklearn.preprocessing import LabelEncoder


# Sample DataFrame : Exemplo com 3 formas geometricas com diferentes tamanhos e cores
colunas=['NOME', 'INGLES_DESC', 'CURSOU_MAT1_DESC']

exemplo = df_data.iloc[:4].loc[:, colunas]

print("Exemplo DataFrame: \n\n", exemplo)

# Using LabelEncoder (sklearn)
df_le = pd.DataFrame()
le = LabelEncoder()
for col in exemplo.columns:
    df_le[col + '_le'] = le.fit_transform(exemplo[col])
print('*************************************************************')
print("\n\nLabelEncoder DataFrame: \n\n", df_le)


# Using get_dummies to encode multiple columns
df_encoded_multiple = pd.get_dummies(exemplo, columns=['NOME', 'INGLES_DESC', 'CURSOU_MAT1_DESC'])
print('*************************************************************')
print("\n\n Pandas get_dummies Encoded colunas:\n", df_encoded_multiple)

# Dropping the first dummy column to avoid multicollinearity
df_encoded_drop_first = pd.get_dummies(exemplo, columns=['NOME', 'INGLES_DESC', 'CURSOU_MAT1_DESC'], drop_first=True)
print('*************************************************************')
print("\nEncoded with drop_first column to avoid multicollinearity:\n", df_encoded_drop_first)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# LE O DATASET COMO UM PANDAS DATAFRAME.
df_numeros = pd.read_csv('curso_dw.csv')

le = LabelEncoder()

df_numeros['INGLES_DESC'] = le.fit_transform(df_numeros['INGLES_DESC'])
df_numeros['CURSOU_MAT1_DESC'] = le.fit_transform(df_numeros['CURSOU_MAT1_DESC'])
df_numeros['CURSOU_MAT2_DESC'] = le.fit_transform(df_numeros['CURSOU_MAT2_DESC'])
df_numeros['CURSOU_MAT3_DESC'] = le.fit_transform(df_numeros['CURSOU_MAT3_DESC'])
df_numeros['CURSOU_MAT4_DESC'] = le.fit_transform(df_numeros['CURSOU_MAT4_DESC'])
df_numeros['PERFIL'] = le.fit_transform(df_numeros['PERFIL'])


print("\nLabelEncoder DataFrame: \n\n", df_numeros.head())

# Definição das colunas que serão features (nota-se que a coluna NOME não está presente)
features = [
    #"MATRICULA",
    "REPROVACOES_MAT_1", 'REPROVACOES_MAT_2', "REPROVACOES_MAT_3", "REPROVACOES_MAT_4",
    "NOTA_MAT_1", "NOTA_MAT_2", "NOTA_MAT_3", "NOTA_MAT_4",
    "INGLES", "H_AULA_PRES", "TAREFAS_ONLINE", "FALTAS", 
    "INGLES_DESC", # usando todas as colunas
    "CURSOU_MAT1_DESC", "CURSOU_MAT2_DESC", "CURSOU_MAT3_DESC", "CURSOU_MAT4_DESC",
]

# Definição da variável-alvo
target = ["PERFIL"]

# Preparação dos argumentos para os métodos da biblioteca ``scikit-learn``
X = df_numeros[features]
y = df_numeros[target]

# Separação dos dados em um conjunto de treino e um conjunto de teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=337)

In [ ]:
from sklearn import datasets

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

#import autosklearn.classification
import autosklearn.regression


# Parâmetros do AutoML
total_time = 120  # Tempo total (em segundos)
per_model_time = 30  # Tempo máximo por modelo

print("Criando os modelos com AUTOML")
#cls = autosklearn.classification.AutoSklearnClassifier()
reg = autosklearn.regression.AutoSklearnRegressor(
    time_left_for_this_task=total_time,
    per_run_time_limit=per_model_time,
    tmp_folder='/tmp/autosklearn_tmp',
    #output_folder='/tmp/autosklearn_out',
    disable_evaluator_output=False,
    n_jobs=-1
)

# Iniciando barra de progresso em segundo plano
progress_thread_reg = threading.Thread(target=show_progress_bar, args=(total_time,))
progress_thread_reg.start()

print("Treinando os modelos com AUTOML")
#cls.fit(X_train, y_train)
reg.fit(X_train, y_train)

print("Verificando resultados dos modelos com AUTOML")
#classification_predictions = cls.predict(X_test)
regression_predictions = reg.predict(X_test)

# Espera barra terminar se for mais rápida
progress_thread_reg.join()

# Resultados
print("\Ranking dos modelos testados: \n")
print(reg.leaderboard())

y_pred = reg.predict(X_test)

#print(f"\Acurácia: {accuracy_score(y_test, y_pred):.4f}") # essa métrica só faz sentido para classificação

# Avaliar o modelo com diferentes métricas de regressão
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'R²: {r2}')


In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Resultados
print("\Ranking dos modelos testados: \n")
print(reg.leaderboard())


y_pred = reg.predict(X_test)

#print(f"\Acurácia: {accuracy_score(y_test, y_pred):.4f}") # essa métrica só faz sentido para classificação

# Avaliar o modelo com diferentes métricas de regressão
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'R²: {r2}')

In [ ]:
# Create a new MLflow Experiment
mlflow.set_experiment("MLflow AUTOML CLASSIFICACAO x REGRESSAO")

# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    #mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("accuracy", accuracy_score(y_test, regression_predictions)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "AUTOML CLASSIFICACAO x REGRESSAO")

    # Infer the model signature
    signature = infer_signature(X_train, reg.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=reg,
        artifact_path="curso_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="automl-curso-perfil-alunos",
    )

In [ ]:
# Load the model back for predictions as a generic Python Function model
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

predictions = loaded_model.predict(X_test)

#iris_feature_names = datasets.load_iris().feature_names

result = pd.DataFrame(X_test, columns=features)
result["actual_class"] = y_test
result["predicted_class"] = predictions

#result[:4]